<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))
  
Last update: September, 2023

In [3]:
!pip install pylev

In [4]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher

import numpy as np
import pandas as pd

import pylev

In [5]:
pd.set_option("display.max_columns", None)

## 1. Loading the data

In [6]:
homicides_df = pd.read_csv("./data/homicides.csv")

In [7]:
homicides_df.shape

(12400, 22)

In [8]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [10]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4744,05/24/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Domingo,12:30,URB. RIVERAS DE OCCIDENTE ENO REPORTADO8,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1995.0,65-535,solbhk5399@gmail.com
7838,08/21/2021 12:00:00 AM,CAUCA,POPAYÁN (CT),Viernes,0:13,VALLE DEL ORTIGAL,URBANA,APARTAMENTO,ARMA DE FUEGO,A PIE,A PIE,41.0,FEMENINO,CASADO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,19001000.0,1974.0,95-962,bnsnul3698@unidatos.edu.co
11360,12/06/2021 12:00:00 AM,CUNDINAMARCA,FUSAGASUGÁ,Domingo,22:30,CARLOS LLERAS,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,27.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,25290000.0,1988.0,17-762,bapnnu7022@unidatos.edu.co
8082,08/29/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Sábado,3:26,BARRIO COLON CNO REPORTADO10,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,AFRODESCENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1972.0,48-580,paannr9731@gmail.com
11880,12/20/2021 12:00:00 AM,ATLÁNTICO,BARRANQUILLA (CT),Domingo,12:15,EL BOSQUE,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,A PIE,18.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,8001000.0,1997.0,41-673,lmstut1842@unidatos.edu.co


## 2. Working with datetimes

In [2]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [11]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: time data '13/12/2021 12:00:00 AM' does not match format '%m/%d/%Y %H:%M:%S %p'

In [13]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [14]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [16]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [17]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [24]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

AttributeError: Can only use .dt accessor with datetimelike values

*The homicide hour is available in a different column!*

In [25]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11])+homicides_df["HORA"]

KeyError: 'HORA'

In [22]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

KeyError: "['HORA'] not found in axis"

In [ ]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [163]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [26]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [27]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
6201,2021-07-02 6:50,VALLE,YUMBO,Jueves,CGTO MULALO E25,RURAL,CARRETERA,ARMA DE FUEGO,A PIE,A PIE,15.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,76892000.0,0.0,60-818,bndobu9664@gmail.com
8342,2021-09-06 8:45,ANTIOQUIA,COPACABANA,domungo,MACHADO,URBANA,VIAS PUBLICAS,CONTUNDENTES,A PIE,A PIE,81.0,MASCULINO,VIUDO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,5212000.0,1934.0,64-414,qctcnd3108@unidatos.edu.co
2775,2021-03-26 12:45,ATLÁNTICO,BARRANQUILLA (CT),Jueves,VILLA SAN CARLOS,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,PASAJERO MOTOCICLETA,43.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1972.0,55-392,basnod6731@gmail.com
8899,2021-09-22 23:55,VALLE,CALI (CT),Mates,ALTO JORDAN E18,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,32.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1983.0,57-157,gtmcog8608@unidatos.edu.co
3666,2021-04-21 20:17,ANTIOQUIA,RIONEGRO,Martes,SECTOR GALERIA,URBANA,CAFETERIAS,ARMA DE FUEGO,A PIE,A PIE,39.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5615000.0,1976.0,32-611,jashjl4886@unidatos.edu.co


## 2. Fixing categorical column

In [28]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [29]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

In [30]:
homicides_df["DEPARTAMENTO"] = homicides_df["DEPARTAMENTO"].str.replace("SAN ANDRÉS", "SAN ANDRÉS Y PROVIDENCIA", regex=True)
homicides_df["DEPARTAMENTO"] = homicides_df["DEPARTAMENTO"].str.replace("N. DE SANTANDER", "NORTE DE SANTANDER", regex=True)

In [32]:
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 6:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 9:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,2021-12-31 15:42,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12395,2021-12-31 19:04,VALLE,LA CUMBRE,Jueves,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,correo4352@colombia.gov.co
12396,2021-12-31 22:35,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


## 3. Analyzing potential duplicates

In [33]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [34]:
duplicates_by_cedula.shape

(1627, 21)

In [35]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 3:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 5:30,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 3:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 2:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 6:20,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [36]:
duplicates_by_cedula = duplicates_by_cedula.drop_duplicates(subset=["CÉDULA"])

In [37]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
3810,2021-04-26 3:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 5:30,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
138,2021-01-02 6:20,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co
2554,2021-03-19 10:05,NARIÑO,IPIALES,Jueves,CHAMPAGTH,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,39.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52356000.0,1976.0,10-327,kltnur7139@gmail.com
10550,2021-11-14 16:05,CALDAS,SALAMINA,Sábado,VDA LA LOMA,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,18.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,17653000.0,1997.0,10-531,nkgqkf5128@unidatos.edu.co
1361,2021-02-09 21:30,ANTIOQUIA,CIUDAD BOLÍVAR,lune,LA FLORESTA PARTE BAJA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,17.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5101000.0,1998.0,10-722,correo187@unidatos.edu.co


## 4. Fixing formats

In [38]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [39]:
cedula_malformed.shape

(28, 21)

In [40]:
cedula_malformed.head(4)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co


In [41]:
def corregir_cedula(cedula):
    if re.match(r'\d{2}-\d{3}', cedula):
        return cedula  
    else:
        cedula_corregida = re.sub(r'(\d{2})(\d{3})', r'\-1\-2', cedula)
        return cedula_corregida

In [42]:
homicides_df['CÉDULA'] = homicides_df['CÉDULA'].apply(corregir_cedula)

In [43]:
homicides_df['CÉDULA']

0        42-908
1        15-183
2        84-786
3        31-289
4        66-363
          ...  
12394    39-658
12395    39-442
12396    25-368
12397    62-735
12398    47-126
Name: CÉDULA, Length: 12395, dtype: object

<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [44]:
correo_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match(r'^\d{4}@(\w+\.(edu\.co|com))$', x) is None))]

In [45]:
correo_malformed

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 6:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 9:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,2021-12-31 15:42,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12395,2021-12-31 19:04,VALLE,LA CUMBRE,Jueves,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,correo4352@colombia.gov.co
12396,2021-12-31 22:35,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


## 5. Automating imputation of categorical values

In [46]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [47]:
# Calculating the distance between two words using the Levenshtein distance
pylev.levenshtein("sábado", "sabaod")

3

In [48]:
pylev.levenshtein("sábado", "viernes")

7

In [49]:
SequenceMatcher(None, "sábado", "sabaod").ratio()

0.6666666666666666

In [50]:
SequenceMatcher(None, "sábado", "viernes").ratio()

0.15384615384615385

<span style="color:red">How does SequenceMatcher works? How this differ from the Levenshtein distance?</span>

In [51]:
#SequenceMatcher se utiliza para medir la similitud entre secuencias y proporciona información detallada 
#sobre las similitudes y diferencias, mientras que la distancia de Levenshtein se utiliza 
#para medir la diferencia entre dos cadenas de texto calculando el número mínimo de operaciones
#necesarias para convertir una en otra. 

<span style="color:red">TODO: Create a function to fix the digitation errors for the "DIA" column.</span>

In [52]:
homicides_df.reset_index(drop=True, inplace = True)

def corregir_dias(df, col):

    dias_correctos = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

    ajuste =[]

    for index, row in df.iterrows():
      mejor_ratio = 0

      for i in dias_correctos:
        matches = SequenceMatcher(None,row[col], i).ratio()

        if matches > mejor_ratio:
          mejor_ratio = matches
          value = i

      ajuste.append(value)

    col_final = pd.DataFrame(ajuste)

    return col_final

homicides_df['dias_corregidos'] = corregir_dias(homicides_df, 'DIA')

In [53]:
homicides_df['dias_corregidos']

0        Jueves
1        Jueves
2        Jueves
3        Jueves
4        Jueves
          ...  
12390    Jueves
12391    Jueves
12392    Jueves
12393    Jueves
12394    Jueves
Name: dias_corregidos, Length: 12395, dtype: object

## 6. Recalculation based on a different column

In [54]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
10949,1964.0,51.0
6942,1983.0,32.0
10588,1991.0,24.0
3970,1995.0,20.0
10623,1975.0,40.0
163,1979.0,36.0
7711,1978.0,37.0
5550,1973.0,42.0
3560,1988.0,27.0
1345,1956.0,59.0


In [55]:
def fix_age(row):
    current_year = datetime.now().year
    row["AÑO DE NACIMIENTO"] = int(current_year - row["EDAD"])
    return row

homicides_df = homicides_df.apply(fix_age, axis=1)





In [56]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
8515,1993,30.0
11817,1965,58.0
856,1977,46.0
8112,1976,47.0
9658,1989,34.0
11006,1994,29.0
966,1999,24.0
6934,2002,21.0
1032,1991,32.0
3450,1988,35.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

## 7. Conclusion

<span style="color:red">Make a summary of the different data quality problems found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating that specific problem.</span>

In [446]:
# Problemas de Calidad

# Dentro del set de datos entregado, encontramos problemas de calidad relacionados con datos incompletos, 
# algunos errores de entrada durante la captura, igualmente, se detectaron problemas en estandarización de datos
# y presencia de registros duplicados en la base de datos, lo que puede llevar a la redundancia y la inconsistencia
# en los datos. 

# Dimensión de Calidad de Datos

# Dentro de los aspectos que fueron encontrados relacionados con la dimensión de calidad de datos, encontramos 
# inconvenientes con los temas de exactitud, integridad y completitud de la información. Otro de los temas relacionados, 
# es la unicidad que se relaciona con la presencia de duplicados en el conjunto de datos.

# Estrategia Implementada

# La estrategia ejecutada para asegurar la calidad de los datos, se centró en varias técnica de limpieza, iniciando con
# la eliminación de información con problemas de estandarización, posteriormente, se ejecutó una tarea para eliminar 
# datos duplicados en un campo clave de las consultas que era la identificación de la persona, con esto asegurábamos 
# el listado de personas únicos y relacionados por departamento estandarizado. Con el fin de contar con información 
# integra, se corrigieron los temas de formato de la cedula, continuando con el aseguramiento de la identificación de 
# las personas. Continuando con las herramientas para estandarización y limpieza, se aseguran los datos de contacto
# e información relevante sobre los días de los homicidios y edad de las personas relacionadas en el dataset.
